In [1]:
# Import des librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
import json
import xml

In [2]:
import pymongo

In [3]:
import xmltodict
from tqdm import tqdm

In [4]:
from datetime import datetime, date

In [5]:
folder = 'D:/Fichiers_dev/test_xml/'

In [6]:
tree = ET.parse(folder + 'declarations.xml')

In [7]:
root = tree.getroot()

In [17]:
declarations_json = []

for declaration in tqdm(root) :
    # pour chaque declaration (1er niveau du parsage), on reconvertie en xml string decodé en utf-8
    declaration_parse = ET.tostring(declaration).decode('utf-8')
    # on convertie en json chaque declaration
    declaration_json = xmltodict.parse(declaration_parse)
    # on enregistre chaque json dans une liste
    declarations_json.append((declaration_json['declaration']))

100%|███████████████████████████████████████████████████████████████████████████| 10893/10893 [00:32<00:00, 330.13it/s]


In [18]:
def remplacer_espace_insecable(dictionnaire, cle, valeur):
    
    # si la valeur est une chaine, faire le remplacement
    dictionnaire[cle] = valeur.replace('\xa0', '')

In [19]:
def conversion(dictionnaire, cle, valeur):
    
    # conversion en numerique
    if (cle=='annee' or cle=='montant'): 
        dictionnaire[cle] = int(valeur)
    
    # conversion en datetime
    if cle=='dateDepot':
        format_date = '%d/%m/%Y %H:%M:%S'
        dictionnaire[cle] = datetime.strptime(valeur, format_date)
    
    # conversion en date
    if cle=='dateNaissance':
        format_date = '%d/%m/%Y'
        # on recupere seulement la date de l'objet datetime
        dictionnaire[cle] = datetime.strptime(valeur, format_date)

In [20]:
def transformation_dictionnaire_recursive_fonction(dictionnaire, fonction):
    for cle, valeur in dictionnaire.items():

        # si la valeur est un dictionnaire, appel récursif
        if isinstance(valeur, dict):
                valeur_dict = valeur
                transformation_dictionnaire_recursive_fonction(dictionnaire=valeur_dict, fonction=fonction)

        # si la valeur est une liste, on boucle sur la liste, puis appel récusrsif
        if isinstance(valeur, list):
                for dictionnaire_in in valeur :
                    transformation_dictionnaire_recursive_fonction(dictionnaire=dictionnaire_in, fonction=fonction)

        if isinstance(valeur, str) :
            fonction(dictionnaire, cle, valeur)

        else : 
            dictionnaire[cle] = valeur

In [21]:
# on applique les fonctions de transformation à toutes les déclarations

for declaration in tqdm(declarations_json):
    # remplacer les espaces insécables
    transformation_dictionnaire_recursive_fonction(dictionnaire=declarations_json[0], fonction=remplacer_espace_insecable)
    # faire les conversions numériques
    transformation_dictionnaire_recursive_fonction(dictionnaire=declarations_json[0], fonction=conversion)

100%|██████████████████████████████████████████████████████████████████████████| 10893/10893 [00:03<00:00, 3002.20it/s]


In [22]:
declarations_json[0]

{'dateDepot': datetime.datetime(2022, 7, 11, 15, 40, 13),
 'uuid': '4344aaa1-874d-4e6d-9b1a-45f7725b710c',
 'origine': 'ADEL',
 'complete': 'true',
 'attachedFiles': {'attachedFiles': {'fileName': 'VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML',
   'serverFileName': None,
   'base64EncodedContent': None}},
 'declarationVersion': '20171221',
 'activConsultantDto': {'neant': 'true'},
 'activProfCinqDerniereDto': {'neant': 'true'},
 'activProfConjointDto': {'items': {'items': {'motif': {'id': 'CREATION',
     'label': None},
    'commentaire': None,
    'nomConjoint': '[Données non publiées]',
    'employeurConjoint': 'CENTRE HOSPITALIER DU HAUT-BUGEY',
    'activiteProf': 'Infirmière'}},
  'neant': 'false'},
 'fonctionBenevoleDto': {'neant': 'true'},
 'mandatElectifDto': {'items': {'items': [{'motif': {'id': 'CREATION',
      'label': None},
     'commentaire': 'REVENUS NETS IMPOSABLES\n        [Données non publiées]',
     'descriptionMandat': 'DEPUTE',
     'remuneration': {'brutNet': 'Net',
     

### Connection à MongoDB

In [23]:
client = pymongo.MongoClient('mongodb://localhost:27017')

In [24]:
db = client['declarations']

collection = db['declarations_all']

### Conversion en json et transfert dans MongoDB

In [25]:
collection.drop()

for declaration in tqdm(declarations_json):
    collection.insert_one(declaration)

100%|██████████████████████████████████████████████████████████████████████████| 10893/10893 [00:05<00:00, 1851.49it/s]
